In [1]:
from dotenv import load_dotenv
import os
import time
import json
import requests

### CREDENTIAL CONFIGURATION

In [2]:
load_dotenv(override=True)
azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
api_key = os.getenv('AZURE_OPENAI_API_KEY')
api_version = os.getenv('API_VERSION')
assistant_id = os.getenv('ASSISTANT_ID')
model = os.getenv('MODEL')
params = {
    "api-version": api_version
}

headers = {
    "api-key" : api_key
}

### APIs EXPLORATION

#### 1. POST /threads API --> CREATE NEW THREAD

In [5]:
def thread_api(azure_endpoint=azure_endpoint, params=params, headers=headers):
    thread_api = azure_endpoint + "openai/threads"
    response = requests.post(thread_api, params=params, headers=headers)
    return response.json()['id']

thread_id = thread_api()
thread_id

'thread_kGGNSlnotvVPqJk4ILCOBBsH'

#### 2. POST /messages API --> ADD MESSAGES TO CHAT

In [3]:
def add_messages(azure_endpoint=azure_endpoint, thread_id=None, params=params, headers=headers, messages : str = 'hello'):
    message_api = azure_endpoint + "openai/threads/" + thread_id + "/messages"

    body = {
        "role": "user",
        "content": messages
        }
    
    response = requests.post(message_api, 
                                 params=params, 
                                 headers=headers,
                                json=body)

    if response.status_code == 200:
        # Parse the JSON response (assuming the API returns JSON data)
        data = response.json()
        return data
    else:
        # If the request failed, print the status code and the response text
        print(f"Request failed with status code {response.status_code}")
        return response.text

print(add_messages())

TypeError: can only concatenate str (not "NoneType") to str

#### 3. POST /runs API --> PUSH MESSAGES TO CHATGPT ENGINE

In [6]:
def runs_thread(azure_endpoint=azure_endpoint, thread_id=None, params=params, headers=headers, assistant_id=None, model=model):
    run_api = azure_endpoint + "openai/threads/" + thread_id + "/runs"
    body = {
        "assistant_id": assistant_id,
        "model": model
        }

    response = requests.post(run_api,params=params,headers=headers,json=body)

    if response.status_code == 200:
        # Parse the JSON response (assuming the API returns JSON data)
        data = response.json()
        return data
    else:
        # If the request failed, print the status code and the response text
        print(f"Request failed with status code {response.status_code}")
        return response.text

print(runs_thread())

{'id': 'run_QD1q08M8VUWOTGZYnQp34YXP', 'object': 'thread.run', 'created_at': 1723530531, 'assistant_id': 'asst_qccNPjxTUX1AlabUDNxCEbbQ', 'thread_id': 'thread_LGcQrmB7xTWy69fR19WZRrsB', 'status': 'queued', 'started_at': None, 'expires_at': 1723531131, 'cancelled_at': None, 'failed_at': None, 'completed_at': None, 'required_action': None, 'last_error': None, 'model': 'gpt-4o-mini', 'instructions': '### Description: ###\nThe Buzzebees SKU Matching Agent is designed to streamline the process of matching products from purchase orders or invoice documents with the corresponding products in the Buzzebees database. The primary function of this agent is to handle discrepancies in SKU codes by comparing product names, which are predominantly in Thai, and identifying the most similar matches. The agent utilizes natural language processing techniques to analyze and compare the textual content of product names, providing a structured output in JSON format that includes the matched product, its SKU

#### 4. GET /messages API --> REQUEST CHAT HISTORY

In [18]:
def get_chat_history(azure_endpoint=azure_endpoint, thread_id=None, params=params, headers=headers):
    message_api = azure_endpoint + "openai/threads/" + thread_id + "/messages"
    response = requests.get(message_api, params=params, headers=headers)
    # Check if the request was successful
    if response.status_code == 200:
            # Parse the JSON response (assuming the API returns JSON data)
            data = response.json()

            # Check if the response came from assistant or not
            if (data["data"][0]["role"] != 'assistant') or (len(data["data"][0]["content"]) == 0) :
                time.sleep(1)
                get_chat_history()
            else:
                return data
            
    else:
        # If the request failed, print the status code and the response text
        print(f"Request failed with status code {response.status_code}")
        return response.text



**GET ONLY LATEST CHAT**

In [9]:
get_chat_history()["data"][0]["content"][0]['text']['value']

'Hello! How can I assist you today?'

### EXTRACT JSON FROM RESPONSE

In [10]:
full_response = get_chat_history()["data"][6]
full_response

{'id': 'msg_gfQQSAw53uX8wgJM8q2HceXL',
 'object': 'thread.message',
 'created_at': 1723364684,
 'assistant_id': 'asst_qccNPjxTUX1AlabUDNxCEbbQ',
 'thread_id': 'thread_LGcQrmB7xTWy69fR19WZRrsB',
 'run_id': 'run_5M4upouk6W9DXr3cLotXmrl5',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': {'value': 'Hi again! How can I help you today?',
    'annotations': []}}],
 'attachments': [],
 'metadata': {}}

In [11]:
full_response['content'][0]['text']['value']

'Hi again! How can I help you today?'

In [13]:
def extract_json(response):
    return json.loads(response.split('```')[1].replace('\n', '').replace('json', ''))

### REQUEST MATCHING DATA

**CREATE PROMPT**

In [14]:
from prompt import GetPrompt

default_prompt = GetPrompt('vendor_data', 'bzbs_data').prompt
print(default_prompt)


                Here is the invoice data that is read from vendor invoice:
                -- VENDOR DATA STARTS HERE --

                vendor_data

                -- VENDOR DATA ENDS HERE --

                And, here is the product data from Buzzebees database:
                -- BUZZEBEES DATA STARTS HERE --

                bzbs_data

                -- BUZZEBEES DATA ENDS HERE --

                Please provide the matching result in JSON format.
                


**MAIN FUNCTION**

In [26]:
def request_match(azure_endpoint=azure_endpoint, thread_id=None,assistant_id=None, vendor_data : str=None, bzbs_data : str=None):

    thread_id = thread_api()
    
    prompt = GetPrompt(vendor_data, bzbs_data).prompt
    
    add_messages(thread_id=thread_id,messages=prompt)

    runs_thread(thread_id=thread_id, assistant_id=assistant_id)

    response = get_chat_history()["data"][0]["content"][0]['text']['value']

    result = extract_json(response)

    return result